In [1]:
%matplotlib inline
import utils
import skutils
import pandas as pd

TRAIN_DATA = "../../data/train.csv"
TEST_DATA = "../../data/test.csv"

In [2]:
# Load the training data
train = pd.read_csv(TRAIN_DATA)

# Load the testing data
test = pd.read_csv(TEST_DATA)

In [7]:
def transform(data):
    
    cols_emtpy = [
        'ind_var2', 'ind_var2_0',
        'saldo_var27', 'num_var27', 'ind_var27', 'num_var27_0', 'ind_var27_0',
        'saldo_var28', 'num_var28', 'ind_var28', 'num_var28_0', 'ind_var28_0',
        'saldo_var41', 'num_var41', 'ind_var41',
        'saldo_var46', 'num_var46', 'ind_var46', 'ind_var46_0', 'num_var46_0',
        'imp_amort_var18_hace3',
        'imp_amort_var34_hace3',
        'imp_reemb_var13_hace3', 'num_reemb_var13_hace3',
        'imp_reemb_var33_hace3', 'num_reemb_var33_hace3',
        'num_trasp_var33_out_hace3', 'imp_trasp_var33_out_hace3',
        'imp_trasp_var17_out_hace3', 'num_trasp_var17_out_hace3',
        'num_var2_0_ult1', 'num_var2_ult1',
        'saldo_var2_ult1',
        'saldo_medio_var13_medio_hace3',
    ]
    
    cols_correlated = [
        'ind_var1', 'ind_var1_0',
        'ind_var5',
        'ind_var6', 'ind_var6_0', 'num_var6',
        'ind_var8', 'ind_var8_0',
        'ind_var12',
        'ind_var13_corto', 'ind_var13_corto_0', 'ind_var13_medio', 'ind_var13_medio_0',
        'ind_var18', 'ind_var18_0', 'num_var18', 'num_var18_0',
        'ind_var20', 'ind_var20_0',
        'ind_var29', 'ind_var29_0',
        'ind_var24', 'ind_var24_0',
        'ind_var25_0', 'num_var25_0',
        'ind_var26_0', 'num_var26_0',
        'ind_var32_0', 'num_var32_0',
        'ind_var34', 'ind_var34_0', 'num_var34', 'num_var34_0',
        'ind_var37_0', 'num_var37_0',
        'ind_var39',
        'ind_var40', 'num_var40',
        'ind_var44',
        'num_var29_0',
        'num_var13_medio', 'num_var13_medio_0',
        'num_var7_emit_ult1',
        'num_var39',
        'num_var40_0',
        'num_var44_0',
        'saldo_var6',
        'saldo_var18',
        'saldo_var13_medio',
        'saldo_medio_var17_ult3',
        'saldo_medio_var33_ult1',
        'delta_imp_aport_var13_1y3', 'delta_imp_reemb_var13_1y3', 'delta_num_reemb_var13_1y3',
        'delta_imp_aport_var17_1y3', 'delta_imp_reemb_var17_1y3', 'delta_num_trasp_var17_in_1y3', 'delta_num_trasp_var17_out_1y3',
        'delta_imp_aport_var33_1y3', 'delta_imp_reemb_var33_1y3', 'delta_num_reemb_var33_1y3', 'delta_num_trasp_var33_out_1y3',
        'delta_imp_compra_var44_1y3', 'delta_imp_venta_var44_1y3',
        'delta_imp_trasp_var17_in_1y3', 'delta_imp_trasp_var17_out_1y3',
        'delta_imp_trasp_var33_in_1y3', 'delta_imp_trasp_var33_out_1y3',
        'imp_reemb_var17_hace3',
        'imp_reemb_var33_ult1', 'imp_trasp_var33_out_ult1',
        'num_med_var45_ult3',
        'imp_op_var39_efect_ult1',
        'imp_op_var39_efect_ult3',
        'imp_op_var39_ult1',
        'imp_amort_var18_ult1',
        'saldo_medio_var17_hace3', 'saldo_medio_var17_ult1',
        'saldo_medio_var13_medio_ult3',
    ]
    
    cols_irrelevant = [
        'ind_var7_emit_ult1',
        'num_trasp_var17_in_ult1',
        'imp_venta_var44_hace3',
        'imp_aport_var17_hace3',
        'num_var6_0',
        'num_venta_var44_hace3',
        'saldo_medio_var33_ult3',
        'imp_trasp_var17_in_ult1',
        'imp_trasp_var33_in_ult1',
        'num_trasp_var17_out_ult1',
        'num_trasp_var33_out_ult1',
        'num_aport_var17_hace3',
        'imp_trasp_var17_out_ult1',
        'num_aport_var33_ult1',
        'num_op_var40_hace3',
        'imp_var7_emit_ult1',
        'saldo_medio_var29_ult3',
        'num_aport_var33_hace3',
        'num_var29',
        'num_meses_var29_ult3',
        'imp_aport_var33_hace3',
        'saldo_var34',
        'saldo_medio_var29_ult1',
        'saldo_medio_var33_hace3',
        'saldo_medio_var29_hace2',
        'saldo_medio_var13_medio_ult1',
        'delta_num_trasp_var33_in_1y3',
        'saldo_var29',
        'num_trasp_var33_in_ult1',
        'num_trasp_var33_in_hace3',
        'saldo_var20',
        'saldo_medio_var29_hace3',
        'imp_trasp_var33_in_hace3',
        'num_reemb_var17_hace3',
        'num_meses_var13_medio_ult3',
        'imp_aport_var33_ult1',
        'delta_imp_amort_var34_1y3',
        'imp_trasp_var17_in_hace3',
        'imp_amort_var34_ult1',
        'delta_imp_amort_var18_1y3',
        'num_reemb_var33_ult1',
        'num_trasp_var17_in_hace3',
        'saldo_medio_var13_medio_hace2',
        'ind_var10cte_ult1',
        'num_var5_0',
        'ind_var9_ult1',
        'ind_var10_ult1',
        'ind_var9_cte_ult1',
        'saldo_medio_var8_ult3',
        'num_meses_var13_corto_ult3',
        'saldo_var8',
        'saldo_medio_var8_ult1',
        'ind_var26_cte',
        'num_var24',
        'num_var12_0',
        'num_var8',
        'saldo_medio_var8_hace2',
        'saldo_var26',
        'num_var24_0',
        'num_var12',
        'saldo_var25',
        'ind_var25_cte',
        'num_sal_var16_ult1',
        'saldo_medio_var12_ult1',
        'saldo_medio_var13_corto_hace2',
        'saldo_medio_var12_ult3',
        'num_var1_0',
        'saldo_medio_var13_corto_ult3',
        'saldo_medio_var13_corto_ult1',
        'ind_var40_0',
        'saldo_var13_corto',
        'saldo_medio_var12_hace2',
        'num_var26',
        'ind_var26',
        'num_var25',
        'saldo_var13',
        'saldo_var12',
        'num_var13_corto_0',
        'saldo_var24',
        'ind_var14_0',
        'imp_sal_var16_ult1',
        'ind_var25',
        'ind_var30_0',
        'num_var13_corto',
        'num_var14_0',
        'var21',
        'ind_var7_recib_ult1',
        'num_var13',
        'saldo_medio_var8_hace3',
        'num_var7_recib_ult1',
        'num_var13_0',
        'imp_var7_recib_ult1',
        'imp_op_var40_efect_ult1',
        'ind_var13_largo_0',
        'imp_aport_var13_hace3',
        'saldo_medio_var12_hace3',
        'ind_var32_cte',
        'ind_var19',
        'imp_op_var40_efect_ult3',
        'delta_num_aport_var13_1y3',
        'saldo_var40',
        'num_op_var40_efect_ult1',
        'num_op_var40_efect_ult3',
        'num_var1',
        'num_var20_0',
        'saldo_var1',
        'ind_var31_0',
        'num_aport_var13_hace3',
        'ind_var13_largo',
        'ind_var14',
        'ind_var31',
        'num_meses_var13_largo_ult3',
        'imp_op_var40_comer_ult3',
        'saldo_medio_var13_corto_hace3',
        'imp_op_var40_ult1',
        'num_op_var40_ult1',
        'num_op_var40_comer_ult3',
        'imp_op_var40_comer_ult1',
        'num_var14',
        'num_op_var40_ult3',
        'num_var20',
        'imp_aport_var13_ult1',
        'num_op_var40_comer_ult1',
        'num_var31_0',
        'num_var13_largo',
        'num_var32',
        'saldo_var32',
        'saldo_var14',
        'ind_var17_0',
        'num_var13_largo_0',
        'ind_var32',
        'imp_compra_var44_ult1',
        'num_reemb_var17_ult1',
        'num_var31',
        'delta_num_compra_var44_1y3',
        'num_op_var40_hace2',
        'num_meses_var17_ult3',
        'num_aport_var13_ult1',
        'imp_reemb_var17_ult1',
        'saldo_var13_largo',
        'ind_var17',
        'num_var17_0',
        'saldo_var31',
        'ind_var44_0',
        'num_var17',
        'num_aport_var17_ult1',
        'num_var44',
        'saldo_var44',
        'num_meses_var44_ult3',
        'saldo_var17',
        'imp_reemb_var13_ult1',
        'saldo_medio_var13_largo_ult3',
        'saldo_medio_var44_ult3',
        'saldo_medio_var13_largo_hace2',
        'num_reemb_var13_ult1',
        'imp_aport_var17_ult1',
        'saldo_medio_var13_largo_ult1',
        'ind_var33_0',
        'delta_num_aport_var17_1y3',
        'num_compra_var44_ult1',
        'ind_var33',
        'saldo_medio_var44_ult1',
        'num_var33_0',
        'delta_num_reemb_var17_1y3',
        'num_meses_var33_ult3',
        'saldo_medio_var44_hace2',
        'delta_num_venta_var44_1y3',
        'saldo_medio_var17_hace2',
        'num_var33',
        'saldo_medio_var33_hace2',
        'imp_venta_var44_ult1',
        'num_venta_var44_ult1',
        'delta_num_aport_var33_1y3',
        'saldo_medio_var13_largo_hace3',
        'saldo_medio_var44_hace3',
        'imp_compra_var44_hace3',
        'saldo_var33',
        'num_compra_var44_hace3',
    ]
    
    # Extract the ids
    ids = data['ID']
    
    # Extract the data
    X = data.drop(['ID'] + cols_emtpy + cols_correlated + cols_irrelevant,  axis=1)
    if 'TARGET' in data.columns:
        X.drop('TARGET', axis=1, inplace=True)

    # Extract the labels
    y = data['TARGET'] if 'TARGET' in data.columns else None
    
    return ids, X, y

# Create Test Dataset

In [21]:
from sklearn.cross_validation import StratifiedShuffleSplit

# Transform the training data
Ids_train, X, y = transform(train)

sss = StratifiedShuffleSplit(y, random_state=2, n_iter=30, test_size=0.15)

for train_index, test_index in sss:
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]

# Cross Validate XGB

In [13]:
from sklearn.pipeline import Pipeline
import xgboost as xgb

params = {
    'max_depth': 5,
    'n_estimators': 250,
    'learning_rate': 0.05,
    'seed': 42,
}

fit_params = {
    'eval_metric': 'auc',
    
}

clf = xgb.XGBClassifier(**params)

In [23]:
from sklearn.metrics import make_scorer
from sklearn.metrics import roc_auc_score

num_round = 4
seed = 42

# Needs latest XGBoost
# https://github.com/dmlc/xgboost/blob/master/doc/build.md
scores = xgb.cv(params, xgb.DMatrix(X_train, y_train), num_round,  metrics={'auc'},
       folds=skutils.folds(y_train, n_folds=10, stratified=True, random_state=seed), seed=seed)

print("Cross validation test-auc-mean score %.8f (+- %.8f)" 
      % (scores['test-auc-mean'].mean(), scores['test-auc-mean'].std()))
print("Cross validation test-auc-std score %.8f (+- %.8f)" 
      % (scores['test-auc-std'].mean(), scores['test-auc-std'].std()))
print("Cross validation train-auc-mean %.8f (+- %.8f)" 
      % (scores['train-auc-mean'].mean(), scores['train-auc-mean'].std()))
print("Cross validation train-auc-std %.8f (+- %.8f)" 
      % (scores['train-auc-std'].mean(), scores['train-auc-std'].std()))

Cross validation test-auc-mean score 0.81482785 (+- 0.00456677)
Cross validation test-auc-std score 0.01550207 (+- 0.00144960)
Cross validation train-auc-mean 0.82997997 (+- 0.00394259)
Cross validation train-auc-std 0.00199749 (+- 0.00057121)


# Compute Private Score

In [22]:
from sklearn.metrics import roc_auc_score

gbm = clf.fit(X_train.values, y_train.values, eval_metric="auc")

score = roc_auc_score(y_test.values, gbm.predict_proba(X_test.values)[:,1])

print("Private Score %.8f" % score)

Private Score 0.83010656


# Make Submission

In [9]:
Ids_test_, X_test_, y_test_ = transform(test)

gbm = clf.fit(X.values, y.values, eval_metric="auc")

y_pred = gbm.predict_proba(X_test_.values)

submission = pd.DataFrame({"ID":Ids_test_, "TARGET":y_pred[:,1]})
submission.to_csv("submission_%s.csv" % utils.timestamp(), index=False)

# Public Score: 0.837660